In [ ]:
from google.colab import drive
drive.mount('/content/drive')
EXP_NAME = 'hp2_12251652'


Mounted at /content/drive


In [ ]:
M_context_prompt_format = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert in generating quizzes.
Given the context regarding biology, create a single multiple-choice question that accurately tests knowledge based on the context.

Context: {0}

If you decide to provide a quiz, you MUST strictly follow the format below:
The format for the multiple-choice quiz is:

  Question: <Your Question Here>
  Options:
  <Option A>
  <Option B>
  <Option C>
  <Option D>
  Answer: <Correct Option>

Additional rules:
1. Include exactly four options in the \"Options\" list.
2. Avoid repeating any information across the options.
3. Ensure that one and only one option is correct.
4. Ensure that the question and options are directly related to the provided context.
5. You SHOULD NOT include any other text in the response.
<|eot_id|><|start_header_id|>user<|end_header_id|>

Now create a single multiple-choice question.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

S_context_prompt_format = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert in generating quizzes.
Given the context regarding biology, create a single short-answer question that accurately tests knowledge based on the context.

Context: {0}

If you decide to provide a quiz, you MUST strictly follow the format below:

Question: <Your Question Here>
Answer: <Correct Answer>

Additional rules:
1. Ensure the question and answer are directly related to the provided context.
2. Include only one correct answer.
3. You SHOULD NOT include any other text in the response.
<|eot_id|><|start_header_id|>user<|end_header_id|>

Now create a single short-answer question.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

In [ ]:
import os
import numpy as np
import pandas as pd
all_df =[]

json_path = '/content/drive/MyDrive/[개인] YouniB/서강대학교-일반화학1_수정.json'
tmp_df = pd.read_json(json_path)
tmp_df = tmp_df[['Question','Answer','type','Options','Context','Page','과목명','강의명']]
tmp_df['Options'] = tmp_df['Options'].replace({'nan': np.nan, 'NaN': np.nan, 'NAN': np.nan}, regex=True)
all_df.append(tmp_df)

df = pd.concat(all_df)

print(f'전체 df의 객관식, 단답식 분포')
display(df['type'].value_counts())

전체 df의 객관식, 단답식 분포


,count
type,
객관식,62
단답식,61


In [ ]:
!pip install transformers -qq
! pip install accelerate  -qq
! pip install tiktoken -qq
! pip install einops  -qq
! pip install scipy  -qq
! pip install transformers_stream_generator -qq
! pip install peft -qq
! pip install deepspeed  -qq
! pip install bitsandbytes -qq
! pip install trl -qq
!pip install auto-gptq optimum -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
! pip install -U bitsandbytes


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

load_path = f"<BASE DIRECTODY>/FT/merged_model_{EXP_NAME}"

model = AutoModelForCausalLM.from_pretrained(
    load_path,
    device_map='auto',
    low_cpu_mem_usage=True
    )
tokenizer = AutoTokenizer.from_pretrained(load_path)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
! nvidia-smi

Wed Dec 25 19:31:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   50C    P0              29W /  72W |   5631MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import torch

# 1. Load Data
def prepare_data(df):
    data = []
    for idx, row in df.iterrows():
        if row['type'] == '객관식':
            prompt_template = M_context_prompt_format
        elif row['type'] == '단답식':
            prompt_template = S_context_prompt_format
        else:
            print(f'{idx}번째 행의 Type 오류')

        # context와 프롬프트 분리
        context = row['Context']
        prompt = prompt_template.format(context)

        if row['type'] == '객관식':
            output_text = (
                f"Question: {row['Question']}\n"
                "Options:\n"
                f"  {row['Options'][0]}\n"
                f"  {row['Options'][1]}\n"
                f"  {row['Options'][2]}\n"
                f"  {row['Options'][3]}\n"
                f"Answer: {row['Answer']}"
            )
        else:
            output_text = f"Question: {row['Question']}\nAnswer: {row['Answer']}"

        data.append({"input_text": prompt, "context": context, "output_text": output_text})

    return data

data = prepare_data(df)
dataset = Dataset.from_pandas(pd.DataFrame(data))

#############################################################################

# 2. Prompt Prefix 길이 미리 계산
def calculate_prompt_length(prompt_template):
    placeholder_context = " "
    full_prompt = prompt_template.format(placeholder_context)
    tokenized_prompt = tokenizer(
        full_prompt,
        truncation=False,
        add_special_tokens=False
    )["input_ids"]
    return len(tokenized_prompt)


# Prompt Format별로 고정된 길이 계산
M_prompt_length = calculate_prompt_length(M_context_prompt_format)
S_prompt_length = calculate_prompt_length(S_context_prompt_format)
print(f'M_prompt_length: {M_prompt_length}')
print(f'S_prompt_length: {S_prompt_length}')

#############################################################################

# 3. Preprocess Function
def preprocess_data(examples):
    MAX_TOKENS = 3200

    model_inputs = {
        "input_ids": [],
        "attention_mask": [],
        "labels": [],
        'context_text': []
    }

    for input_text in examples["input_text"]:

        # Context가 삽입된 위치를 파악
        context_start_index = input_text.find("Context: ") + len("Context: ")
        if context_start_index == -1:
            raise ValueError("Prompt format is invalid. 'Context:' not found.")

        # 프롬프트 앞부분과 뒷부분 분리
        prompt_prefix = input_text[:context_start_index]  # "Context:" 포함
        context_start_index = context_start_index  # Context의 실제 텍스트 시작 위치
        context_end_index = input_text.find("Now create", context_start_index)
        if context_end_index == -1:
            raise ValueError("Prompt format is invalid. 'Now create' not found after context.")

        # Context 추출
        context = input_text[context_start_index:context_end_index].strip()
        prompt_suffix = input_text[context_end_index:].strip()

        # 프롬프트 앞/뒤의 고정 길이를 계산
        prompt_prefix_ids = tokenizer(
            prompt_prefix,
            truncation=False,
            add_special_tokens=False
        )["input_ids"]
        prompt_suffix_ids = tokenizer(
            prompt_suffix,
            truncation=False,
            add_special_tokens=False
        )["input_ids"]

        fixed_prompt_length = len(prompt_prefix_ids) + len(prompt_suffix_ids)

        # Context에 할당된 최대 길이 계산
        max_context_length = MAX_TOKENS - fixed_prompt_length

        # Context 토큰화 및 잘라내기
        context_ids = tokenizer(
            context,
            truncation=True,
            max_length=max_context_length,
            padding=False,
            add_special_tokens=False
        )["input_ids"]

        # 최종 프롬프트 병합
        full_prompt_ids = prompt_prefix_ids + context_ids + prompt_suffix_ids
        attention_mask = [1] * len(full_prompt_ids)

        # 입력과 출력 병합
        combined_ids = full_prompt_ids
        combined_mask = attention_mask

        # 라벨 생성
        labels = [-100] * len(full_prompt_ids)

        # 결과 저장
        model_inputs["input_ids"].append(combined_ids)
        model_inputs["attention_mask"].append(combined_mask)
        model_inputs["labels"].append(labels)
        model_inputs['context_text'].append(context)

    return model_inputs

tokenized_dataset = dataset.map(preprocess_data, batched=True)


M_prompt_length: 185
S_prompt_length: 123


Map:   0%|          | 0/123 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader

class CustomDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features):
        input_ids = [feature['input_ids'] for feature in features]
        attention_mask = [feature['attention_mask'] for feature in features]
        context_text = [feature['context_text'] for feature in features]
        input_text = [feature['input_text'] for feature in features]

        # 패딩 적용
        batch = self.tokenizer.pad(
            {'input_ids': input_ids, 'attention_mask': attention_mask},
            padding=True,
            return_tensors="pt"
        )
        # context_text와 output_text를 그대로 batch에 추가
        batch['context_text'] = context_text
        batch['input_text'] = input_text
        return batch

data_collator = CustomDataCollator(tokenizer)

test_dataloader = DataLoader(
    tokenized_dataset,
    batch_size=16,
    collate_fn=data_collator
)

for i, x in enumerate(test_dataloader):
    print(i)
    for k, v in x.items():
        print(k)
        if k in ['input_ids','attention_mask']:
            print(v)
            print(f"각 dataloader에 들어있는 키({k}) 토큰 개수: {', '.join(str(len(q)) for q in v)} -- DATA COLLATE FN")
        else:
            print(v)
        print()
    print()
    print()
    if i == 2:
        break


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


0
input_ids
tensor([[128000, 128006,   9125,  ..., 128006,  78191, 128007],
        [128000, 128006,   9125,  ..., 128006,  78191, 128007],
        [128000, 128006,   9125,  ..., 128009, 128009, 128009],
        ...,
        [128000, 128006,   9125,  ..., 128009, 128009, 128009],
        [128000, 128006,   9125,  ..., 128009, 128009, 128009],
        [128000, 128006,   9125,  ..., 128009, 128009, 128009]])
각 dataloader에 들어있는 키(input_ids) 토큰 개수: 553, 553, 553, 553, 553, 553, 553, 553, 553, 553, 553, 553, 553, 553, 553, 553 -- DATA COLLATE FN

attention_mask
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
각 dataloader에 들어있는 키(attention_mask) 토큰 개수: 553, 553, 553, 553, 553, 553, 553, 553, 553, 553, 553, 553, 553, 553, 553, 553 -- DATA COLLATE FN

context_text
['Classification of Matter Based on Composition If you follow t

In [ ]:
import transformers
transformers.__version__

'4.47.1'

In [ ]:
import time
from datetime import datetime


In [ ]:
import gc
import torch
import time

generated_outputs = []
all_context = []
all_input_text = []

for batch_idx, batch in enumerate(test_dataloader):
    print(f"\nProcessing batch {batch_idx}")
    print(f"  Before batch {batch_idx}: Allocated: {torch.cuda.memory_allocated()} Reserved: {torch.cuda.memory_reserved()}")
    start_time = time.time()

    input_ids = batch['input_ids'].to(model.device)
    attention_mask = batch['attention_mask'].to(model.device)
    print(f"   Batch {batch_idx}: input_ids size: {input_ids.size()}, attention_mask size: {attention_mask.size()}")

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=192,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.75,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"      Start:time: {datetime.now()}\n      Elapsed time: {elapsed_time:.2f} seconds")

    # 생성된 텍스트 디코딩 (배치 단위)
    for sample_idx, output in enumerate(outputs):
        input_length = input_ids[sample_idx].shape[0]
        decoded_output = tokenizer.decode(output[input_length:], skip_special_tokens=True)

        generated_outputs.append(decoded_output.strip())
        all_context.append(batch['context_text'][sample_idx])
        all_input_text.append(batch['input_text'][sample_idx])

        if sample_idx == 0:
            print(tokenizer.decode(output,skip_special_tokens=True))

    print(f"  After batch {batch_idx}: Allocated: {torch.cuda.memory_allocated()} Reserved: {torch.cuda.memory_reserved()}\n\n")
    del input_ids, attention_mask, outputs
    gc.collect()
    torch.cuda.empty_cache()



Processing batch 0
  Before batch 0: Allocated: 5702771200 Reserved: 5704253440
   Batch 0: input_ids size: torch.Size([16, 553]), attention_mask size: torch.Size([16, 553])


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


      Start:time: 2024-12-25 19:31:43.619996
      Elapsed time: 37.44 seconds
system
You are an expert in generating quizzes.
Given the context regarding biology, create a single multiple-choice question that accurately tests knowledge based on the context.

Context: Classification of Matter Based on Composition If you follow this scheme you can determine how to classify any type of matter Homogeneous mixture Heterogeneous mixture Element Compound Classification of MatterSubstances A substance has distinct properties and a composition that does not vary from sample to sample The two types of substances are elements and compounds An element is a substance which can not be decomposed to simpler substances A compound is a substance which can be decomposed to simpler substances Compounds and Composition Compounds have a definite composition That means that the relative number of atoms of each element that makes up the compound is the same in any sample This is The Law of Constant Composit

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



Processing batch 1
  Before batch 1: Allocated: 5711374336 Reserved: 5725224960
   Batch 1: input_ids size: torch.Size([16, 496]), attention_mask size: torch.Size([16, 496])
      Start:time: 2024-12-25 19:32:14.564685
      Elapsed time: 30.60 seconds
system
You are an expert in generating quizzes.
Given the context regarding biology, create a single multiple-choice question that accurately tests knowledge based on the context.

Context: Whats Ahead 11 THE STUDY OF CHEMISTRY We begin with a brief description of what chemistry is what chemists do and why it is useful to learn chemistry 12 CLASSIFICATIONS OF MATTER Next we examine some fundamental ways to classify matter distinguishing between pure substances and mixtures and between elements and compounds 13PROPERTIESOFMATTER We then consider different characteristics or properties used to characterize identify and separate substances 14 UNITS OF MEASUREMENT We observe that many properties rely on quantitative measurements involving n

...


Processing batch 5
  Before batch 5: Allocated: 5711387136 Reserved: 5725224960
   Batch 5: input_ids size: torch.Size([16, 1043]), attention_mask size: torch.Size([16, 1043])
      Start:time: 2024-12-25 19:35:58.552228
      Elapsed time: 57.79 seconds
system
You are an expert in generating quizzes.
Given the context regarding biology, create a single multiple-choice question that accurately tests knowledge based on the context.

Context: 22 THE DISCOVERY OF ATOMIC STRUCTURE Discovery of the Nucleus Ernest Rutherford shot particles at a thin sheet of gold foil and observed the pattern of scatter of the particles Figure 29 Rutherfords scattering experiment 22 THE DISCOVERY OF ATOMIC STRUCTURE The Nuclear Atom Since some particles were deflected at large angles Thompsons model could not be correct Figure 210 The structure of the atom A cloud of rapidly moving electrons occupies most of the volume of the atom The nucleus occupies a tiny region at the center of the atom and is composed 

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



Processing batch 6
  Before batch 6: Allocated: 5711441920 Reserved: 5725224960
   Batch 6: input_ids size: torch.Size([16, 926]), attention_mask size: torch.Size([16, 926])
      Start:time: 2024-12-25 19:36:44.572688
      Elapsed time: 45.66 seconds
system
You are an expert in generating quizzes.
Given the context regarding biology, create a single multiple-choice question that accurately tests knowledge based on the context.

Context: Periodic Table The periodic table is a systematic catalog of the elements Elements are arranged in order of atomic number Periodic Table The rows on the periodic chart are periods Columns are groups Elements in the same group have similar chemical properties Coinage metals Cu Ag and Au belong to group 11 or 1B Less reactive than most metals Periodicity Many elements show very strong similarities to one another When one looks at the chemical properties of elements one notices a repeating pattern of reactivities Figure 213 Arranging elements by atomic 

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



Processing batch 7
  Before batch 7: Allocated: 5711423488 Reserved: 5725224960
   Batch 7: input_ids size: torch.Size([11, 948]), attention_mask size: torch.Size([11, 948])
      Start:time: 2024-12-25 19:37:25.893728
      Elapsed time: 40.97 seconds
system
You are an expert in generating quizzes.
Given the context regarding biology, create a single multiple-choice question that accurately tests knowledge based on the context.

Context: History Democritus and Greek philosophers BC 400 The material world must be made up of tiny indivisible particles Atomos indivisible or uncuttable Plato and Aristotle There can be no ultimately indivisible particles The atomic view of matter faded for many centuries The notion of atoms reemerged in Europe during the 17th century Chemists learned to measure the amounts of elements that reacted with one another to form new substances The ground was laid for an atomic theory that linked the idea of elements with the idea of atoms 21 THE ATOMIC THEORY OF

In [ ]:
results_df = pd.DataFrame({
    'input': all_input_text,
    'context': all_context,
    'generated': generated_outputs,
})
display(results_df.info())
results_df.to_csv(f'/content/drive/MyDrive/[개인] YouniB/sehyeon/결과분석/result_{EXP_NAME}.csv',index=False)
results_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   input      123 non-null    object
 1   context    123 non-null    object
 2   generated  123 non-null    object
dtypes: object(3)
memory usage: 3.0+ KB


None

,input,context,generated
0,<|begin_of_text|><|start_header_id|>system<|en...,Classification of Matter Based on Composition ...,Question: What is a characteristic of a compou...
1,<|begin_of_text|><|start_header_id|>system<|en...,Classification of Matter Based on Composition ...,Question: What is a substance that cannot be d...
2,<|begin_of_text|><|start_header_id|>system<|en...,Classification of Matter Based on Composition ...,Question: Classify a sample of air as either a...
3,<|begin_of_text|><|start_header_id|>system<|en...,Classification of Matter Based on Composition ...,Question: What is a characteristic that distin...
4,<|begin_of_text|><|start_header_id|>system<|en...,Classification of MatterMixtures Mixtures exhi...,Question: What is another name for a homogeneo...
...,...,...,...
118,<|begin_of_text|><|start_header_id|>system<|en...,EXERCISE 211 Determining the Formula of an Oxy...,Question: What is the name of the acid derived...
119,<|begin_of_text|><|start_header_id|>system<|en...,21 THE ATOMIC THEORY OF MATTER Daltons Postula...,Question: What does Dalton's Atomic Theory sta...
120,<|begin_of_text|><|start_header_id|>system<|en...,21 THE ATOMIC THEORY OF MATTER Daltons Postula...,Question: What is a key characteristic of the ...
121,<|begin_of_text|><|start_header_id|>system<|en...,21 THE ATOMIC THEORY OF MATTER Daltons Postula...,Question: What is the main idea behind Dalton'...


In [ ]:
import pandas as pd
results_df = pd.read_csv(f'<BASE DIRECTORY>/결과분석/result_{EXP_NAME}.csv')
results_df

,input,context,generated
0,<|begin_of_text|><|start_header_id|>system<|en...,Classification of Matter Based on Composition ...,Question: What is a characteristic of a compou...
1,<|begin_of_text|><|start_header_id|>system<|en...,Classification of Matter Based on Composition ...,Question: What is a substance that cannot be d...
2,<|begin_of_text|><|start_header_id|>system<|en...,Classification of Matter Based on Composition ...,Question: Classify a sample of air as either a...
3,<|begin_of_text|><|start_header_id|>system<|en...,Classification of Matter Based on Composition ...,Question: What is a characteristic that distin...
4,<|begin_of_text|><|start_header_id|>system<|en...,Classification of MatterMixtures Mixtures exhi...,Question: What is another name for a homogeneo...
...,...,...,...
118,<|begin_of_text|><|start_header_id|>system<|en...,EXERCISE 211 Determining the Formula of an Oxy...,Question: What is the name of the acid derived...
119,<|begin_of_text|><|start_header_id|>system<|en...,21 THE ATOMIC THEORY OF MATTER Daltons Postula...,Question: What does Dalton's Atomic Theory sta...
120,<|begin_of_text|><|start_header_id|>system<|en...,21 THE ATOMIC THEORY OF MATTER Daltons Postula...,Question: What is a key characteristic of the ...
121,<|begin_of_text|><|start_header_id|>system<|en...,21 THE ATOMIC THEORY OF MATTER Daltons Postula...,Question: What is the main idea behind Dalton'...


In [ ]:
results_df.isnull().sum()

,0
input,0
context,0
generated,0


In [ ]:
#print(results_df.iloc[0]['original'])
#print('='*100)
print(results_df.iloc[1]['context'])
print('='*100)
print(results_df.iloc[1]['input'])
print('='*100)
print(results_df.iloc[1]['generated'])
print('='*100)
print('='*100)
print('='*100)



Classification of Matter Based on Composition If you follow this scheme you can determine how to classify any type of matter Homogeneous mixture Heterogeneous mixture Element Compound Classification of MatterSubstances A substance has distinct properties and a composition that does not vary from sample to sample The two types of substances are elements and compounds An element is a substance which can not be decomposed to simpler substances A compound is a substance which can be decomposed to simpler substances Compounds and Composition Compounds have a definite composition That means that the relative number of atoms of each element that makes up the compound is the same in any sample This is The Law of Constant Composition or The Law of Definite Proportions Classification of MatterMixtures Mixtures exhibit the properties of the substances that make them up Mixtures can vary in composition throughout a sample heterogeneous or can have the same composition throughout the sample homogen

In [ ]:
! pip install opik



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 100.7 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [ ]:
import opik
import os
os.environ["OPIK_API_KEY"]="<OPIK API KEY>"
opik.configure(use_local=False)

Do you want to use "1000century" workspace? (Y/n)y


OPIK: Configuration saved to file: /root/.opik.config


In [ ]:
print(len(results_df.iloc[0]['input']))

3051


In [ ]:
results_df

,input,context,generated
0,<|begin_of_text|><|start_header_id|>system<|en...,Classification of Matter Based on Composition ...,Question: What is a characteristic of a compou...
1,<|begin_of_text|><|start_header_id|>system<|en...,Classification of Matter Based on Composition ...,Question: What is a substance that cannot be d...
2,<|begin_of_text|><|start_header_id|>system<|en...,Classification of Matter Based on Composition ...,Question: Classify a sample of air as either a...
3,<|begin_of_text|><|start_header_id|>system<|en...,Classification of Matter Based on Composition ...,Question: What is a characteristic that distin...
4,<|begin_of_text|><|start_header_id|>system<|en...,Classification of MatterMixtures Mixtures exhi...,Question: What is another name for a homogeneo...
...,...,...,...
118,<|begin_of_text|><|start_header_id|>system<|en...,EXERCISE 211 Determining the Formula of an Oxy...,Question: What is the name of the acid derived...
119,<|begin_of_text|><|start_header_id|>system<|en...,21 THE ATOMIC THEORY OF MATTER Daltons Postula...,Question: What does Dalton's Atomic Theory sta...
120,<|begin_of_text|><|start_header_id|>system<|en...,21 THE ATOMIC THEORY OF MATTER Daltons Postula...,Question: What is a key characteristic of the ...
121,<|begin_of_text|><|start_header_id|>system<|en...,21 THE ATOMIC THEORY OF MATTER Daltons Postula...,Question: What is the main idea behind Dalton'...


In [ ]:
opik_input_text = """You are an expert in generating quizzes.
Given the context regarding biology, create a single multiple-choice question that accurately tests knowledge based on the context.
If you decide to provide a quiz, you MUST strictly follow the format below:
The format for the multiple-choice quiz is:

  Question: <Your Question Here>
  Options:
  <Option A>
  <Option B>
  <Option C>
  <Option D>
  Answer: <Correct Option>

Additional rules:
1. Include exactly four options in the \"Options\" list.
2. Avoid repeating any information across the options.
3. Ensure that one and only one option is correct.
4. Ensure that the question and options are directly related to the provided context.
5. You SHOULD NOT include any other text in the response."""

In [ ]:
from datetime import datetime
now = datetime.now()
formatted_time = now.strftime("%H%M")
formatted_time

'1937'

In [ ]:
import opik
from opik import Opik
from opik.evaluation import evaluate
from opik.evaluation.metrics import Hallucination
import os
opik.configure(use_local=False)
os.environ["OPENAI_API_KEY"] = "<OPEN API KEY>"

# Opik 클라이언트 생성 + 새 데이터셋 생성
client = Opik()
dataset = client.get_or_create_dataset(name=f"QA generation_{EXP_NAME}_{formatted_time}")

# 데이터 추가
result_list = []
for _, row in results_df.iterrows():
    result_list.append(
        {
            'input':opik_input_text,
            'context':row['context'],
            'generated':row['generated'],
        }
    )
#dataset.insert(result_list)



OPIK: Opik is already configured. You can check the settings by viewing the config file at /root/.opik.config
OPIK: Created a "QA generation_hp2_12251652_1937" dataset at https://www.comet.com/opik/1000century/redirect/datasets?name=QA%20generation_hp2_12251652_1937.


In [ ]:
batch_size = 1
for i in range(0, len(result_list), batch_size):
    batch = result_list[i:i+batch_size]
    try:
        dataset.insert(batch)
        print(f"Successfully inserted batch {i//batch_size + 1}")
    except Exception as e:
        print(batch)
        print(f"Error inserting batch {i//batch_size + 1}: {str(e)}")


Successfully inserted batch 1
Successfully inserted batch 2
Successfully inserted batch 3
Successfully inserted batch 4
Successfully inserted batch 5
Successfully inserted batch 6
Successfully inserted batch 7
Successfully inserted batch 8
Successfully inserted batch 9
Successfully inserted batch 10
Successfully inserted batch 11
Successfully inserted batch 12
Successfully inserted batch 13
Successfully inserted batch 14
Successfully inserted batch 15
Successfully inserted batch 16
Successfully inserted batch 17
Successfully inserted batch 18
Successfully inserted batch 19
Successfully inserted batch 20
Successfully inserted batch 21
Successfully inserted batch 22
Successfully inserted batch 23
Successfully inserted batch 24
Successfully inserted batch 25
Successfully inserted batch 26
Successfully inserted batch 27
Successfully inserted batch 28
Successfully inserted batch 29
Successfully inserted batch 30
Successfully inserted batch 31
Successfully inserted batch 32
Successfully inse

In [ ]:
def evaluation_task(x):
    return {
        "output": x['generated']
    }

# Define the metrics
hallucination_metric = Hallucination(model='gpt-4o-mini')

evaluation = evaluate(
    experiment_name=f"test_{EXP_NAME}_{formatted_time}",
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=[hallucination_metric],
    experiment_config={
        "model": f'llama-3.1-8b-it-{EXP_NAME}',
        "description": "Hallucination 평가"
    }
)

Evaluation: 100%|██████████| 122/122 [00:14<00:00,  8.26it/s]


╭─ QA generation_hp2_12251652_1937 (122 samples) ─╮
│                                                 │
│ Total time:        00:00:15                     │
│ Number of samples: 122                          │
│                                                 │
│ hallucination_metric: 0.1639 (avg)              │
│                                                 │
╰─────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=618296;https://www.comet.com/opik/1000century/experiments/0193ff53-72dd-7d88-bbe2-020c3a049503/compare?experiments=%5B%220676c5f3-a42b-7bdf-8000-48cc353bca58%22%5D\in your Opik dashboard]8;;\.

In [ ]:
import pandas as pd
from pandas import json_normalize

# 기존 데이터프레임
eval_df = pd.DataFrame(evaluation.test_results)


In [ ]:
eval_df


,test_case,score_results
0,{'trace_id': '0676c5f3-bbe1-7c54-8000-676da92e...,"[{'name': 'hallucination_metric', 'value': 0.0..."
1,{'trace_id': '0676c5f3-bcb6-7e89-8000-bbeef6e5...,"[{'name': 'hallucination_metric', 'value': 0.2..."
2,{'trace_id': '0676c5f3-bc17-72e7-8000-4c19cdb8...,"[{'name': 'hallucination_metric', 'value': 0.0..."
3,{'trace_id': '0676c5f3-bc0c-7796-8000-17987ff7...,"[{'name': 'hallucination_metric', 'value': 1.0..."
4,{'trace_id': '0676c5f3-bba2-7d78-8000-928c3e4e...,"[{'name': 'hallucination_metric', 'value': 0.0..."
...,...,...
117,{'trace_id': '0676c5f4-851a-75e0-8000-a14498c3...,"[{'name': 'hallucination_metric', 'value': 0.0..."
118,{'trace_id': '0676c5f4-82db-73aa-8000-d968fc6f...,"[{'name': 'hallucination_metric', 'value': 0.0..."
119,{'trace_id': '0676c5f4-8afb-741f-8000-b2252d49...,"[{'name': 'hallucination_metric', 'value': 0.0..."
120,{'trace_id': '0676c5f4-8a16-7c25-8000-1988ef64...,"[{'name': 'hallucination_metric', 'value': 0.0..."


In [ ]:
eval_df['hallucination'] = eval_df['score_results'].apply(lambda x: x[0])
eval_df

,test_case,score_results,hallucination
0,{'trace_id': '0676c5f3-bbe1-7c54-8000-676da92e...,"[{'name': 'hallucination_metric', 'value': 0.0...","{'name': 'hallucination_metric', 'value': 0.0,..."
1,{'trace_id': '0676c5f3-bcb6-7e89-8000-bbeef6e5...,"[{'name': 'hallucination_metric', 'value': 0.2...","{'name': 'hallucination_metric', 'value': 0.2,..."
2,{'trace_id': '0676c5f3-bc17-72e7-8000-4c19cdb8...,"[{'name': 'hallucination_metric', 'value': 0.0...","{'name': 'hallucination_metric', 'value': 0.0,..."
3,{'trace_id': '0676c5f3-bc0c-7796-8000-17987ff7...,"[{'name': 'hallucination_metric', 'value': 1.0...","{'name': 'hallucination_metric', 'value': 1.0,..."
4,{'trace_id': '0676c5f3-bba2-7d78-8000-928c3e4e...,"[{'name': 'hallucination_metric', 'value': 0.0...","{'name': 'hallucination_metric', 'value': 0.0,..."
...,...,...,...
117,{'trace_id': '0676c5f4-851a-75e0-8000-a14498c3...,"[{'name': 'hallucination_metric', 'value': 0.0...","{'name': 'hallucination_metric', 'value': 0.0,..."
118,{'trace_id': '0676c5f4-82db-73aa-8000-d968fc6f...,"[{'name': 'hallucination_metric', 'value': 0.0...","{'name': 'hallucination_metric', 'value': 0.0,..."
119,{'trace_id': '0676c5f4-8afb-741f-8000-b2252d49...,"[{'name': 'hallucination_metric', 'value': 0.0...","{'name': 'hallucination_metric', 'value': 0.0,..."
120,{'trace_id': '0676c5f4-8a16-7c25-8000-1988ef64...,"[{'name': 'hallucination_metric', 'value': 0.0...","{'name': 'hallucination_metric', 'value': 0.0,..."


In [ ]:
score_df = json_normalize(eval_df['hallucination'])
score_df


,name,value,reason,metadata,scoring_failed
0,hallucination_metric,0.0,"[""The OUTPUT accurately reflects the informati...",None,False
1,hallucination_metric,0.2,['The correct empirical formula based on the c...,None,False
2,hallucination_metric,0.0,"[""The question accurately reflects a key aspec...",None,False
3,hallucination_metric,1.0,['The OUTPUT incorrectly states that the formu...,None,False
4,hallucination_metric,0.0,"[""The OUTPUT accurately reflects Dalton's thir...",None,False
...,...,...,...,...,...
117,hallucination_metric,0.0,['The OUTPUT accurately reflects the CONTEXT b...,None,False
118,hallucination_metric,0.0,['The output accurately reflects the character...,None,False
119,hallucination_metric,0.0,['The OUTPUT accurately reflects the informati...,None,False
120,hallucination_metric,0.0,['The OUTPUT correctly classifies air as a hom...,None,False


In [ ]:
score_df['value'].describe()

,value
count,122.000000
mean,0.163934
std,0.335272
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
test_case_df = json_normalize(eval_df['test_case'])


In [ ]:
test_case_df

,trace_id,dataset_item_id,scoring_inputs.input,scoring_inputs.generated,scoring_inputs.context,scoring_inputs.output,task_output.output
0,0676c5f3-bbe1-7c54-8000-676da92e8818,0676c5f3-695d-78a5-8000-0fe6639d7697,You are an expert in generating quizzes.\nGive...,Question: What does Dalton's Atomic Theory sta...,21 THE ATOMIC THEORY OF MATTER Daltons Postula...,Question: What does Dalton's Atomic Theory sta...,Question: What does Dalton's Atomic Theory sta...
1,0676c5f3-bcb6-7e89-8000-bbeef6e52a8d,0676c5f3-417f-75cb-8000-b1a573aa4506,You are an expert in generating quizzes.\nGive...,Question: What is the empirical formula of the...,Writing Empirical Formulas Because compounds a...,Question: What is the empirical formula of the...,Question: What is the empirical formula of the...
2,0676c5f3-bc17-72e7-8000-4c19cdb8d8bb,0676c5f3-5550-7cb7-8000-c6f25b9ed3e4,You are an expert in generating quizzes.\nGive...,Question: What is the fundamental concept of m...,History Democritus and Greek philosophers BC 4...,Question: What is the fundamental concept of m...,Question: What is the fundamental concept of m...
3,0676c5f3-bc0c-7796-8000-17987ff79946,0676c5f3-5942-7420-8000-546a5605a4e8,You are an expert in generating quizzes.\nGive...,Question: What is the formula for the selenite...,EXERCISE 211 Determining the Formula of an Oxy...,Question: What is the formula for the selenite...,Question: What is the formula for the selenite...
4,0676c5f3-bba2-7d78-8000-928c3e4e00db,0676c5f3-7541-76fd-8000-9454ac0406f3,You are an expert in generating quizzes.\nGive...,Question: What is the main idea behind Dalton'...,21 THE ATOMIC THEORY OF MATTER Daltons Postula...,Question: What is the main idea behind Dalton'...,Question: What is the main idea behind Dalton'...
...,...,...,...,...,...,...,...
117,0676c5f4-851a-75e0-8000-a14498c3a060,0676c5f1-a5c7-79c4-8000-7428e67cd35e,You are an expert in generating quizzes.\nGive...,Question: What is another name for a homogeneo...,Classification of MatterMixtures Mixtures exhi...,Question: What is another name for a homogeneo...,Question: What is another name for a homogeneo...
118,0676c5f4-82db-73aa-8000-d968fc6f553a,0676c5f1-a9b7-74eb-8000-1bdf576c7571,You are an expert in generating quizzes.\nGive...,Question: What is a characteristic of a mixtur...,Classification of MatterMixtures Mixtures exhi...,Question: What is a characteristic of a mixtur...,Question: What is a characteristic of a mixtur...
119,0676c5f4-8afb-741f-8000-b2252d49d07e,0676c5f1-95ec-76b2-8000-9f0b4d4005f0,You are an expert in generating quizzes.\nGive...,Question: What is a characteristic of a compou...,Classification of Matter Based on Composition ...,Question: What is a characteristic of a compou...,Question: What is a characteristic of a compou...
120,0676c5f4-8a16-7c25-8000-1988ef643930,0676c5f1-9de3-7385-8000-472575caa6af,You are an expert in generating quizzes.\nGive...,Question: Classify a sample of air as either a...,Classification of Matter Based on Composition ...,Question: Classify a sample of air as either a...,Question: Classify a sample of air as either a...
